In [12]:
import sys
import os
import tkinter as tk
from tkinter import Scrollbar, Text, filedialog, messagebox
import numpy as np
import tensorflow as tf
import time
from datetime import datetime
import sqlite3
import spacy
import subprocess
import threading
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tkinter import filedialog
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tkinter import ttk
import cv2
import speech_recognition as sr
from deepface import DeepFace
from PIL import Image, ImageTk
import pyaudio
import pyttsx3
import requests
from fuzzywuzzy import fuzz
from pydub import AudioSegment
from pydub.silence import detect_silence
import noisereduce as nr
import librosa
import soundfile as sf


net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]



# Cargar nombres de clases
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]


nlp = spacy.load("es_core_news_sm")

# Cargar el modelo desde el archivo .h5
model = load_model('mejor_modelo.h5')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Cargar el modelo de emociones
emociones_model = load_model('emotion_model.h5')
emociones_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

API_URL = 'https://magicloops.dev/api/loop/22dfbc51-f14c-4332-a9c5-4e6e7ebc5305/run'

### Paleta de colores ###

COLOR_FONDO_PRINCIPAL = "#E3F2FD"  # Azul claro
COLOR_FONDO_SECUNDARIO = "#90CAF9"  # Azul más oscuro
COLOR_TEXTO_PRINCIPAL = "#212121"  # Gris oscuro
COLOR_BOTONES = "#42A5F5"  # Azul medio
COLOR_BOTONES_HOVER = "#64B5F6"  # Azul más claro
COLOR_BORDES = "#BDBDBD"  # Gris claro

# ==================================================
# Configuración del Chatbot
# ==================================================

class ChatbotApp:
    def __init__(self, window):
        self.window = window
        # Frame del chatbot
        self.chatbot_frame = tk.LabelFrame(window, text="Chatbot de Alzheimer", font = ("Times New Roman", 25), bg=COLOR_FONDO_PRINCIPAL,
            fg=COLOR_TEXTO_PRINCIPAL,
            relief=tk.RAISED,
            borderwidth=2
        )
        self.chatbot_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # Scrollbar
        self.scrollbar = Scrollbar(self.chatbot_frame)
        self.chatbox = Text(
            self.chatbot_frame,
            height=15,
            width=60,
            yscrollcommand=self.scrollbar.set,
            font = ("Times New Roman", 20),
            bg="white",
            fg=COLOR_TEXTO_PRINCIPAL,
            relief=tk.SUNKEN,
            borderwidth=2
            )
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.chatbox.pack(side = tk.TOP, fill = tk.BOTH, expand = True)

### Deshabilitar la edición ###
        self.chatbox.config(state = "disabled")

        ### Configurar etiquetas (tags) para alinear los mensajes ###
        self.chatbox.tag_configure("user",justify = "right", background= "#DCF8C6", relief = "raised", borderwidth = 2)
        self.chatbox.tag_configure("bot", justify="left", background="#ECECEC", relief="raised", borderwidth=2)

        # Campo de entrada y botón de envío
        self.entry_frame = tk.Frame(self.chatbot_frame, bg=COLOR_FONDO_PRINCIPAL)
        self.entry_frame.pack(side = tk.BOTTOM, fill = tk.X, padx = 5, pady = 5)
        
        self.indicativo_label = tk.Label(
            self.entry_frame,
            text = "Escribe tu mensaje aquí: ",
            font = ("Times New Roman", 16),
            bg=COLOR_FONDO_PRINCIPAL,
            fg=COLOR_TEXTO_PRINCIPAL
            )
        self.indicativo_label.pack(side = tk.TOP, anchor = tk.W, pady = (0, 5))

        ### Frame para el campo de entrada y el botón (dentro del entry_frame)###
        self.input_frame = tk.Frame(self.entry_frame, bg=COLOR_FONDO_PRINCIPAL)
        self.input_frame.pack(side = tk.TOP, fill = tk.X)

        self.entry = tk.Entry(
            self.input_frame,
            width=50,
            font = ("Times New Roman", 20),
            bg = "white",
            fg = COLOR_TEXTO_PRINCIPAL,
            relief = tk.SUNKEN,
            borderwidth = 2
            )
        
        self.entry.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        
        
        
        self.send_button = tk.Button(
            self.input_frame,
            text="Enviar",
            command=self.send_message,
            font = ("Times New Roman", 16)
            )
        self.send_button.pack(side = tk.RIGHT)

        # Enviar mensaje al presionar Enter
        self.entry.bind("<Return>", lambda event: self.send_message())

    def send_message(self):
        user_text = self.entry.get().strip()
        if user_text:

            ### Habilitar temporalmente para insertar texto ###
            self.chatbox.config(state = "normal" )

            ### Mostrar la pregunta en el chat (Alineado a la derecha)###
            self.chatbox.insert(tk.END, f"Tú: {user_text}\n", "user")

            ### Desplazar al final del chat###
            self.chatbox.see(tk.END)

            ### Enviar la pregunta a la API ###
            response_text = self.get_response_from_api(user_text)

            ### Mostrar la respuesta en el chat (Alineado a la izquierda)###
            self.chatbox.insert(tk.END, f"Bot: {response_text}\n\n", "bot")

            ### Desplazar al final del chat ###
            self.chatbox.see(tk.END)  

            ### Deshabilitar de nuevo ###
            self.chatbox.config(state = "disabled")

            ### Limpiar el campo de entrada ###
            self.entry.delete(0, tk.END)

    def get_response_from_api(self, question):
        ### Lista de saludos comunes ###
        saludos = ["hola", "holis", "buenos días", "buenas tardes", "buenas noches", "hi", "hello", "que tal", "cómo estás"]

        ### Convertir la pregunta a minúsculas para hacer la comparación insensible a mayúsculas ###
        question_lower = question.lower()

        ### Verificar si la pregunta contiene un saludo (incluso con errores tipográficos) ###
        contiene_saludo = any(fuzz.partial_ratio(saludo, question_lower) > 80 for saludo in saludos)
        
        ### Si contiene un saludo, responder con un saludo amigable ###
        if contiene_saludo:
            respuesta_saludo = "¡Hola! Soy un chatbot diseñado para ayudarte con información sobre el Alzheimer. ¿En qué puedo ayudarte hoy?"

            ### Si la pregunta contiene algo más aparte del saludo, responder también a eso ###
            if len(question_lower.split()) > 2: ### Si hay más de dos palabras, asumimos que hay una pregunta###

        ### Si no es un saludo, enviar la pregunta a la API###

                try:
                    payload = {"question": question}
                    response = requests.get(API_URL, json = payload)

            # Verificar si la respuesta es un JSON válido
                    try:
                        response_json = response.json()  # Intenta convertir la respuesta a JSON
                        
                        if isinstance(response_json, dict):  # Si es un diccionario

                            respuesta_api = response_json.get("response", "Lo siento, no pude obtener una respuesta.")

                        elif isinstance(response_json, list) and len(response_json) > 0:  # Si es una lista
                            respuesta_api = response_json[0].get("response", "Lo siento, no pude obtener una respuesta.")
                        
                        else:
                            # Si la respuesta no es un diccionario ni una lista, devolver el texto directamente
                            respuesta_api = str(response_json)

                    except ValueError:  # Si no se puede convertir a JSON

                        # Si la respuesta no es un JSON válido, devolver el texto directamente
                        respuesta_api = response.text

                except requests.RequestException as e:
                    return f"Error: No se pudo conectar con el servidor. Detalles: {str(e)}"
            
            else:
            # Si solo es un saludo, devolver solo el saludo
                return respuesta_saludo

    # Si no es un saludo, enviar la pregunta a la API
        try:
            payload = {"question": question}
            response = requests.get(API_URL, json=payload)

            # Verificar si la respuesta es un JSON válido
            try:
                response_json = response.json()  # Intenta convertir la respuesta a JSON
                if isinstance(response_json, dict):  # Si es un diccionario
                    return response_json.get("response", "Lo siento, no pude obtener una respuesta.")
                elif isinstance(response_json, list) and len(response_json) > 0:  # Si es una lista
                    return response_json[0].get("response", "Lo siento, no pude obtener una respuesta.")
                else:
                    # Si la respuesta no es un diccionario ni una lista, devolver el texto directamente
                    return str(response_json)
            except ValueError:  # Si no se puede convertir a JSON

    # Si la respuesta no es un JSON válido, devolver el texto directamente
                return response.text

        except requests.RequestException as e:
            return f"Error: No se pudo conectar con el servidor. Detalles: {str(e)}"

# ==================================================
# Configuración del Monitoreo
# ==================================================


def calcular_velocidad_habla(texto, tiempo_transcurrido):
    palabras = texto.split()
    if tiempo_transcurrido > 0:
        velocidad = len(palabras) / tiempo_transcurrido  # Palabras por segundo
        return velocidad
    return 0


def filtrar_ruido(audio_path, output_path="audio_filtrado.wav"):
    # Verificar si el archivo existe
    if not os.path.exists(audio_path):
        print(f"El archivo {audio_path} no existe. Esperando a que se cree...")
        return None  # O puedes retornar un valor por defecto o manejar el error de otra manera

    # Cargar el audio
    audio, rate = librosa.load(audio_path, sr=None)

    # Reducir el ruido
    audio_filtrado = nr.reduce_noise(y=audio, sr=rate)

    # Guardar el audio filtrado
    sf.write(output_path, audio_filtrado, rate)
    return output_path


def detectar_pausas_largas(audio_path, umbral_silencio=-50, duracion_minima_pausa=3000):
    # Cargar el archivo de audio
    audio = AudioSegment.from_file(audio_path)

    # Detectar silencios
    silencios = detect_silence(audio, min_silence_len=duracion_minima_pausa, silence_thresh=umbral_silencio)

    # Calcular la duración total de las pausas
    duracion_total_pausas = sum(fin - inicio for inicio, fin in silencios)
    return duracion_total_pausas / 1000  # Convertir a segundos



def detectar_retraso_en_habla(texto, tiempo_transcurrido, audio_path):
    if not texto:  # Si no hay texto, no hay alerta
        return None

    # Calcular velocidad del habla
    velocidad = calcular_velocidad_habla(texto, tiempo_transcurrido)
    
    # Detectar pausas largas
    duracion_pausas = detectar_pausas_largas(audio_path, umbral_silencio=-45)  # Ajustar umbral de silencio
    
    # Umbrales ajustados
    umbral_velocidad = 0.6  # Menos de 0.6 palabras por segundo indica posible retraso
    umbral_pausas = max(3.5, tiempo_transcurrido * 0.3)  # Pausas largas basadas en duración del audio
    
    # Calcular variabilidad en la velocidad del habla
    palabras = texto.split()
    if len(palabras) > 5:  # Solo calcular si hay suficientes palabras
        tiempos_entre_palabras = [tiempo_transcurrido / len(palabras)] * len(palabras)  # Simulación básica
        variabilidad = np.std(tiempos_entre_palabras)
    else:
        variabilidad = 0

    umbral_variabilidad = 0.4  # Si la variabilidad es alta, hay inconsistencias en el habla
    
    alertas = []
    if velocidad < umbral_velocidad:
        alertas.append(f"Velocidad del habla muy lenta ({velocidad:.2f} palabras/segundo).")
    if duracion_pausas > umbral_pausas:
        alertas.append(f"Pausas largas detectadas ({duracion_pausas:.1f} segundos).")
    if variabilidad > umbral_variabilidad:
        alertas.append(f"Patrón de habla inconsistente detectado (variabilidad: {variabilidad:.2f}).")
    
    if alertas:
        return "Alerta: Posible retraso en el habla. " + " ".join(alertas)
    
    return None


def analizar_fluidez(texto, tiempo_transcurrido):
    palabras = texto.split()
    if len(palabras) < 3:  # Frase muy corta
        return "Alerta: Frase demasiado corta. Posible dificultad para hablar."

    # Detectar repeticiones de palabras exactas o similares
    repeticiones = {}
    for i in range(len(palabras)):
        palabra_actual = palabras[i]
        if palabra_actual in repeticiones:
            repeticiones[palabra_actual] += 1
        else:
            # Verificar si hay palabras similares ya registradas
            for palabra_registrada in repeticiones:
                if fuzz.ratio(palabra_actual, palabra_registrada) > 80:  # Umbral de similitud
                    repeticiones[palabra_registrada] += 1
                    break
            else:
                repeticiones[palabra_actual] = 1

    for palabra, count in repeticiones.items():
        if count > 3:  # Palabra repetida más de 3 veces
            return f"Alerta: Palabra '{palabra}' repetida {count} veces. Posible dificultad para hablar."

    return None

cap = None
camara_encendida = False

class EnhancedCameraApp:
    def __init__(self, window, window_label, alertas_text):
        self.window = window
        self.window_label = window_label
        self.alertas_text = alertas_text
        self.video_source = 0
        self.vid = None
        self.is_camera_on = False
        self.audio = pyaudio.PyAudio()
        self.muted = False
        self.tts_engine = pyttsx3.init()
        self.voices = self.tts_engine.getProperty('voices')
        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()
        self.is_listening = False
        self.setup_ui()


    def on_close(self):
        # Detener el reconocimiento de voz y la cámara al cerrar la ventana
        self.stop_listening()
        self.stop_camera()

    def stop_listening(self):
        if self.is_listening:
            self.is_listening = False  # Detener el hilo de reconocimiento de voz
            self.btn_listen.config(text="Iniciar Reconocimiento")

        # Liberar el micrófono
            if hasattr(self, 'microphone'):
                self.microphone.__exit__(None, None, None)

    def setup_ui(self):
        control_frame = ttk.Frame(self.window, style = "Custom.TFrame")
        control_frame.pack(pady=10)

        self.btn_frame = ttk.Frame(self.window, style = "Custom.TFrame")
        self.btn_frame.pack(pady=10)

        self.btn_start = tk.Button(
            self.btn_frame,
            text="Iniciar Cámara",
            command=self.start_camera,
            font = ("Times New Roman", 16)
            )
        

        self.btn_start.pack(side=tk.LEFT, padx=5)

        self.btn_stop = tk.Button(
            self.btn_frame,
            text="Detener Cámara",
            command=self.stop_camera,
            state=tk.DISABLED,
            font = ("Times New Roman", 16)
            )
        
        self.btn_stop.pack(side=tk.LEFT, padx=5)

        self.btn_mute = tk.Button(
            self.btn_frame,
            text="Silenciar",
            command=self.toggle_mute,
            font = ("Times New Roman", 16)
            )
        self.btn_mute.pack(side=tk.LEFT, padx=5)

        self.btn_listen = tk.Button(
            self.btn_frame,
            text="Iniciar Reconocimiento",
            command=self.toggle_listen,
            font = ("Times New Roman", 16)
            )
        self.btn_listen.pack(side=tk.LEFT, padx=5)

        self.canvas = tk.Canvas(self.window, width=640, height=480, bg = COLOR_FONDO_PRINCIPAL)
        self.canvas.pack()

    def start_camera(self):
        global cap, camara_encendida, monitoreo_activo  # Acceder a las variables globales
        if not self.is_camera_on:
            self.vid = cv2.VideoCapture(self.video_source)
            if not self.vid.isOpened():
                print("Error: No se pudo abrir la cámara.")
                return
            self.is_camera_on = True
            cap = self.vid  # Actualizar la variable global `cap`
            camara_encendida = True  # Actualizar el estado de la cámara
            activar_monitoreo(True)  # Activar el monitoreo
            self.update_camera()
            self.btn_start.config(state=tk.DISABLED)
            self.btn_stop.config(state=tk.NORMAL)

# Modificar la función stop_camera para desactivar el monitoreo
    def stop_camera(self):
        global cap, camara_encendida, monitoreo_activo  # Acceder a las variables globales
        if self.is_camera_on:
            self.is_camera_on = False
            self.vid.release()
            cap = None  # Limpiar la variable global `cap`
            camara_encendida = False  # Actualizar el estado de la cámara
            activar_monitoreo(False)  # Desactivar el monitoreo
            self.canvas.delete("all")
            self.btn_start.config(state=tk.NORMAL)
            self.btn_stop.config(state=tk.DISABLED)

    def toggle_mute(self):
        self.muted = not self.muted
        self.btn_mute.config(text="Activar sonido" if self.muted else "Silenciar")

    def update_camera(self):
        if self.is_camera_on:
            ret, frame = self.vid.read()
            if ret:
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(rgb)
                imgtk = ImageTk.PhotoImage(image=img)
                self.canvas.imgtk = imgtk
                self.canvas.create_image(0, 0, anchor=tk.NW, image=imgtk)
            self.window.after(30, self.update_camera)

    def toggle_listen(self):
        global reconocimiento_voz_activo
        if not self.is_listening:
            self.is_listening = True
            reconocimiento_voz_activo = True  # Activar el reconocimiento de voz
            activar_monitoreo(True)  # Activar el monitoreo
            self.btn_listen.config(text="Detener Reconocimiento")
            threading.Thread(target=self.recognize_speech, daemon=True).start()
        else:
            self.is_listening = False
            reconocimiento_voz_activo = False  # Desactivar el reconocimiento de voz
            activar_monitoreo(False)  # Desactivar el monitoreo
            self.btn_listen.config(text="Iniciar Reconocimiento")
    def recognize_speech(self):
        try:
            with self.microphone as source:
                self.recognizer.adjust_for_ambient_noise(source, duration=2)
                if app_running:
                    self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] Sistema listo para escuchar...\n")
                    self.window.after(0, self.alertas_text.see, tk.END)

                while self.is_listening and app_running:
                    try:
                        inicio = time.time()
                        if app_running:
                            self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] Escuchando...\n")
                            self.window.after(0, self.alertas_text.see, tk.END)

                    # Escuchar el audio desde el micrófono
                        audio = self.recognizer.listen(source, timeout=5)
                        fin = time.time()
                        tiempo_transcurrido = fin - inicio

                    # Guardar el audio en un archivo temporal
                        with open("temp_audio.wav", "wb") as f:
                            f.write(audio.get_wav_data())

                    # Filtrar el ruido del audio
                        audio_filtrado_path = filtrar_ruido("temp_audio.wav")

                    # Transcribir el audio filtrado
                        texto = self.recognizer.recognize_google(audio_filtrado_path, language="es-ES")
                        if app_running:
                            self.window.after(0, self.display_recognized_text, texto)

                    # Analizar fluidez y coherencia
                        alerta_fluidez = analizar_fluidez(texto, tiempo_transcurrido)
                        if alerta_fluidez and app_running:
                            self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] {alerta_fluidez}\n")
                            self.alertas_text.see(tk.END)

                        alerta_retraso_habla = detectar_retraso_en_habla(texto, tiempo_transcurrido, audio)
                        if alerta_retraso_habla and app_running:
                            self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] {alerta_retraso_habla}\n")
                            self.alertas_text.see(tk.END)

                    # Detectar pausas largas
                        duracion_pausas = detectar_pausas_largas(audio_filtrado_path)
                        if duracion_pausas > 4.0:  # Umbral de pausas largas
                            self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] Alerta: Pausas largas detectadas ({duracion_pausas:.1f} segundos).\n")
                            self.alertas_text.see(tk.END)

                    except sr.UnknownValueError:
                        if app_running:
                            self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] No se pudo entender el audio. Por favor, repite.\n")
                            self.window.after(0, self.alertas_text.see, tk.END)
                    except sr.RequestError as e:
                        if app_running:
                            self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] Error en la solicitud al servicio de reconocimiento: {e}\n")
                            self.window.after(0, self.alertas_text.see, tk.END)
                    except sr.WaitTimeoutError:
                        if app_running:
                            self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] Tiempo de espera agotado. Intenta de nuevo.\n")
                            self.window.after(0, self.alertas_text.see, tk.END)
                    except Exception as e:
                        if app_running:
                            self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] Error inesperado: {e}\n")
                            self.window.after(0, self.alertas_text.see, tk.END)
        except Exception as e:
            print(f"Error en recognize_speech: {e}")
            self.window.after(0, self.alertas_text.see, tk.END)


    def display_recognized_text(self, text):
        self.alertas_text.insert(tk.END, f"[Voz Detectada] {text}\n")
        self.alertas_text.see(tk.END)

# Función para detectar actividad física
prev_frame = None

def detectar_personas(frame):
    """Detecta si hay personas en la imagen usando YOLOv4"""
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5 and classes[class_id] == "person":
                return True  # Se detectó una persona

    return False  # No se detectó ninguna persona  # No se detectó ninguna persona


def detectar_movimiento():
    """Detecta movimiento solo si hay una persona en la imagen"""
    global prev_frame, cap
    if not camara_encendida or cap is None:
        return False

    ret, frame = cap.read()
    if not ret:
        return False

    # Verificar si hay personas en la imagen antes de analizar movimiento
    if not detectar_personas(frame):
        return False  # No hay personas, no analizar movimiento

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    if prev_frame is None:
        prev_frame = gray
        return True  # Si es el primer frame, considerar que no hay movimiento

    frame_delta = cv2.absdiff(prev_frame, gray)
    prev_frame = gray
    movimiento = np.sum(frame_delta) > 500000  # Umbral de cambio en píxeles

    return not movimiento




def detectar_emocion():
    global cap
    if not camara_encendida or cap is None:
        return "No se detecta cámara"
    
    ret, frame = cap.read()
    if not ret:
        return "No se pudo capturar imagen"
    
    try:
        # Redimensionar la imagen al tamaño esperado por el modelo (224x224)
        resized = cv2.resize(frame, (224, 224))
        
        # Convertir la imagen a un array y normalizarla
        resized = resized / 255.0
        
        # Expandir las dimensiones para que coincidan con la entrada del modelo
        resized = np.expand_dims(resized, axis=0)
        
        # Predecir la emoción
        predicciones = emociones_model.predict(resized)
        
        # Imprimir las predicciones para depuración
        print("Predicciones:", predicciones)
        
        # Obtener la emoción predicha (índice con la probabilidad más alta)
        emocion_predicha = np.argmax(predicciones, axis=1)[0]
        emociones = ["Enojo", "Feliz", "Tristeza", "Sorpresa", "Neutral"]
        if emocion_predicha >= len(emociones):
            return f"Error: Índice de emoción fuera de rango ({emocion_predicha})"
        
        # Obtener la emoción detectada
        emocion_detectada = emociones[emocion_predicha]
        
        return f"Emoción detectada: {emocion_detectada}"
    
    except Exception as e:
        return f"Error al detectar emoción: {str(e)}"

# Configuración de reconocimiento de voz
recognizer = sr.Recognizer()

def detectar_cambios_habla():
    try:
        with sr.Microphone() as source:
            recognizer.adjust_for_ambient_noise(source)
            try:
                inicio = time.time()  # Registrar el tiempo de inicio
                audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
                fin = time.time()  # Registrar el tiempo de finalización
                tiempo_transcurrido = fin - inicio  # Calcular el tiempo transcurrido

                # Guardar el audio en un archivo temporal
                with open("temp_audio.wav", "wb") as f:
                    f.write(audio.get_wav_data())

                # Transcribir el audio
                texto = recognizer.recognize_google(audio, language="es-ES")
                
                # Analizar fluidez y coherencia del habla
                alerta_fluidez = analizar_fluidez(texto, tiempo_transcurrido)  # Pasar tiempo_transcurrido
                if alerta_fluidez:
                    return alerta_fluidez
                
                # Detectar retraso en el habla
                alerta_retraso_habla = detectar_retraso_en_habla(texto, tiempo_transcurrido, "temp_audio.wav")
                if alerta_retraso_habla:
                    return alerta_retraso_habla
                
                return texto
            except sr.WaitTimeoutError:
                return "Alerta: No se detectó sonido en el tiempo esperado."
            except sr.UnknownValueError:
                return "Alerta: No se detectó habla."
            except sr.RequestError:
                return "Alerta: Error en reconocimiento."
    except OSError:
        return "Alerta: No se detecta micrófono, no se puede realizar monitoreo del habla."

# Variable global para controlar el estado del monitoreo
monitoreo_activo = False
app_running = True
reconocimiento_voz_activo = False
tiempo_inicio_inactividad = None
umbral_inactividad = 20
# Función para activar/desactivar el monitoreo
def activar_monitoreo(estado):
    global monitoreo_activo
    monitoreo_activo = estado
    

# Monitoreo en tiempo real
def monitorear_paciente():
    global app_running, camara_encendida, reconocimiento_voz_activo, tiempo_inicio_inactividad  # Asegurar acceso a la variable global

    while app_running:  # Solo monitorear si la aplicación está en ejecución
        if monitoreo_activo and camara_encendida:  # Solo monitorear si la cámara está encendida
            # Verificar si hay personas en el rango de visión de la cámara
            ret, frame = cap.read() if cap else (False, None)
            hay_personas = detectar_personas(frame) if ret else False

            if not hay_personas:
                # Si no hay personas, emitir alerta de "No se detecta ninguna persona"
                alerta = "Alerta: No se detecta ninguna persona."
                if app_running:
                    alertas_text.config(state=tk.NORMAL)  # Habilitar la edición temporalmente
                    alertas_text.insert(tk.END, f"[{datetime.now()}] {alerta}\n")
                    alertas_text.see(tk.END)
                    alertas_text.config(state=tk.DISABLED)  # Deshabilitar la edición
            else:
                # Si hay personas, verificar la actividad física
                actividad_fisica = detectar_movimiento()
                habla = ""
                if reconocimiento_voz_activo:  # Solo monitorear el habla si el reconocimiento de voz está activo
                    habla = detectar_cambios_habla()
                emocion = detectar_emocion()
                alertas = []

                if actividad_fisica:  # Si no hay movimiento
                    if tiempo_inicio_inactividad is None:
                        tiempo_inicio_inactividad = time.time()  # Iniciar el temporizador
                    else:
                        tiempo_transcurrido = time.time() - tiempo_inicio_inactividad
                        if tiempo_transcurrido >= umbral_inactividad:
                            alertas.append(f"Alerta: Inactividad prolongada detectada ({int(tiempo_transcurrido / 60)} minutos).")
                            tiempo_inicio_inactividad = None  # Reiniciar el temporizador
                else:
                    tiempo_inicio_inactividad = None  # Reiniciar el temporizador si hay movimiento

                # Alertas existentes
                if actividad_fisica and tiempo_inicio_inactividad is not None:
                    alertas.append("Alerta: Baja actividad física detectada.")
                if reconocimiento_voz_activo and "Alerta" in habla:  # Verificar si hay una alerta en el habla
                    alertas.append(habla)
                if reconocimiento_voz_activo and "Alerta" in habla:  # Verificar si hay una alerta en el habla
                    alertas.append(habla)

                alertas.append(emocion)

                if app_running:  # Verificar si la ventana todavía está abierta
                    for alerta in alertas:
                        alertas_text.config(state=tk.NORMAL)  # Habilitar la edición temporalmente
                        alertas_text.insert(tk.END, f"[{datetime.now()}] {alerta}\n")
                        alertas_text.see(tk.END)
                        alertas_text.config(state=tk.DISABLED)  # Deshabilitar la edición

        time.sleep(10)





# Función para predecir
def seleccionar_imagen():
    file_path = filedialog.askopenfilename()
    title="Selecciona una imagen",
    filetypes=(("Archivos de imagen", ".jpg *.jpeg *.png"), ("Todos los archivos", ".*"))
    return file_path

def predecir_imagen(file_path):
    img = tf.keras.preprocessing.image.load_img(file_path, target_size=(150, 150))
    img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    prediccion = model.predict(img)
    clase_predicha = np.argmax(prediccion, axis=1)
    return clase_predicha[0]

def predecir_y_mostrar():
    file_path = seleccionar_imagen()
    if file_path:

        ### Cargar Imágen Seleccionada ###

        img = Image.open(file_path)

        ### Redimensionar la imágen para que quepa en la interfaz ###
        img = img.resize((300, 300), Image.Resampling.LANCZOS)

        img_tk = ImageTk.PhotoImage(img)

        #Mostrar la Imágen en el widget de resultado
        imagen_label.config(image = img_tk)

        ### Mantener una referencia para evitar que la imágen sea eliminada por el recolector de basura###
        imagen_label.image = img_tk
    
    
        if file_path:
            clase_predicha = predecir_imagen(file_path)
        if clase_predicha == 0:
            resultado = "La imagen proporcionada pertenece a una persona que se encuentra en la etapa leve del alzheimer.\n Se sugiere que opte por participar en actividades cognitivas y físicas para estimular el cerebro.\n Consultar regularmente in médico especializado en neurología.\n  Se observa una disminución en el volumen del tejido cerebral, especialmente en áreas como el hipocampo (crucial para la memoria) y la corteza cerebral. Esta atrofia es un indicador clave de la enfermedad de Alzheimer"
        elif clase_predicha == 1:
            resultado = "La imagen proporcionada pertenece a una persona que se encuentra en la etapa avanzada del alzheimer. \n Se observa una atrofia cerebral más pronunciada. \n Los ventrículos están más agrandados y la corteza cerebral muestra una mayor pérdida de volumen.\n Se recomienda evaluaciones médicas de forma frecuente"
        elif clase_predicha == 2:
            resultado = "La imagen proporcionada pertenece a una persona que no posee alzheimer. \n  En general, la estructura cerebral parece estar dentro de los límites normales para la edad de la persona. \n No se observa una atrofia cerebral significativa. El volumen del tejido cerebral parece conservado."
        elif clase_predicha == 3:
            resultado = "La imagen proporcionada pertenece a una persona que puede tener indicios de padecer alzheimer. \n Se observa que el patrón de atrofia es consistente con la enfermedad de Alzheimer en una etapa muy temprana. Por lo tanto, existe una posibilidad de que esta persona padezca de alzheimer "
        else:
            resultado = "La imagen proporcionada es incorrecta."
        
        resultado_text.config(state = tk.NORMAL)
        resultado_text.delete(1.0, tk.END)

# Calcular el número de líneas vacías necesarias para centrar verticalmente
        ### Mostrar el resultado en el widget de texto ###
        resultado_text.config(state = tk.NORMAL)
        resultado_text.delete(1.0, tk.END)  

        # Calcular el número de líneas vacías necesarias para centrar verticalmente
        num_lineas = resultado_text.winfo_height() // resultado_text.dlineinfo("@0,0")[3]  # Altura en líneas
        texto_lineas = resultado.count("\n") + 1  # Número de líneas del texto
        lineas_vacias = (num_lineas - texto_lineas) // 2  # Líneas vacías antes del texto

        # Insertar líneas vacías antes del texto
        resultado_text.insert(tk.END, "\n" * lineas_vacias)

        resultado_text.insert(tk.END, resultado, "centrado")

        # Insertar líneas vacías después del texto
        resultado_text.insert(tk.END, "\n" * lineas_vacias)

        resultado_text.config(state=tk.DISABLED)

# ==================================================
# Interfaz gráfica
# ==================================================

ventana = tk.Tk()
ventana.title("Memory Vision")
ventana.geometry("800x600")
ventana.configure(bg=COLOR_FONDO_PRINCIPAL)


def on_close():
    global app_running, reconocimiento_voz_activo

    # Detener el hilo de monitoreo
    app_running = False

    # Detener el reconocimiento de voz
    if reconocimiento_voz_activo:
        camera_app.stop_listening()  # Detener el reconocimiento de voz
        reconocimiento_voz_activo = False

    # Detener la cámara
    camera_app.stop_camera()  # Detener la cámara

    # Cerrar la ventana
    ventana.destroy()


# Vincular el cierre de la ventana a la función on_close
ventana.protocol("WM_DELETE_WINDOW", on_close)


style = ttk.Style()

style.configure("Custom.TFrame", background=COLOR_FONDO_PRINCIPAL)

style.configure("Large.TButton",
                font = ("Times New Roman", 16)
                )

style.configure("TEntry", font = ("Times New Roman", 16), background = "white", foreground = COLOR_TEXTO_PRINCIPAL)

style.configure("TText", font = ("Times New Roman", 16), background = "white", foreground = COLOR_TEXTO_PRINCIPAL)

style.configure("TLabel", font = ("Times New Roman", 16),  background = COLOR_FONDO_PRINCIPAL, foreground = COLOR_TEXTO_PRINCIPAL)

notebook = ttk.Notebook(ventana)
notebook.pack(fill=tk.BOTH, expand=True)

### Pestaña Monitoreo ###
monitoreo_frame = ttk.Frame(notebook, style = "Custom.TFrame")
notebook.add(monitoreo_frame, text="Monitoreo")

alertas_frame = tk.LabelFrame(
    monitoreo_frame,
    text="Alertas",
    font = ("Times New Roman", 25)
    )
alertas_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

alertas_text = tk.Text(
    alertas_frame,
    height=7,
    width=40,
    font = ("Times New Roman", 20)
    )
alertas_text.pack(fill=tk.BOTH, expand=True)

alertas_text.config(state = "disabled")

camara_frame = ttk.Frame(monitoreo_frame)
camara_frame.pack(pady=5)

camara_label = ttk.Label(camara_frame)
camara_label.pack()

camera_app = EnhancedCameraApp(camara_frame, camara_label, alertas_text)

### Pestaña Chatbot ###
chatbot_frame = tk.Frame(notebook)
notebook.add(chatbot_frame, text="Chatbot")

chatbot_system = ChatbotApp(chatbot_frame)

### Pestaña Predicción ###
prediccion_frame = ttk.Frame(notebook)
notebook.add(prediccion_frame, text="Predicción")

pred_frame = tk.LabelFrame(prediccion_frame, text="Predicción de la imagen", font = ("Times New Roman", 25))
pred_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

predecir_button = tk.Button(pred_frame, text="Seleccionar Imagen y Predecir", command=predecir_y_mostrar, font = ("Times New Roman", 16))
predecir_button.pack(pady=5)

imagen_label = tk.Label(pred_frame)
imagen_label.pack(pady = 10)

resultado_text = tk.Text(pred_frame, height=25, width=200, font = ("Times New Roman", 20))
resultado_text.pack(padx=10, pady=70)

resultado_text.tag_configure("centrado", justify="center")

### Habilitar edición temporalmente ###
resultado_text.config(state="normal")

### Volver a deshabilitar ###
resultado_text.config(state="disabled")

# Iniciar monitoreo en segundo plano
monitoreo_thread = threading.Thread(target=monitorear_paciente, daemon=True)
monitoreo_thread.start()
ventana.protocol("WM_DELETE_WINDOW", on_close)
ventana.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicciones: [[0.16659023 0.14677545 0.16814278 0.15393297 0.36455855]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
Predicciones: [[0.16658373 0.14676848 0.16813761 0.15393624 0.36457393]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
Predicciones: [[0.16658668 0.14677243 0.16813809 0.15393773 0.36456504]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Predicciones: [[0.16664326 0.14672238 0.16809472 0.1538975  0.3646421 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
Predicciones: [[0.16664585 0.14673267 0.16810194 0.15390407 0.36461547]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
Predicciones: [[0.16665804 0.14672616 0.16809395 0.15390398 0.3646178 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Predicciones: [[0.16664042 0.14672059 0.16809553 0.15389699 0.3646465 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Predicciones: [[0.16664805 0.14672606 0.16809945 0.15390144 0.364625  ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Predicciones: [[0.16665874 0.14672919 0.16809174 0.1539025  0.364617